# DATA SCIENCE SCHOOL :: Introduction to ML in Python
### An Intensive Python ML Course
## Multinomial Logistic Regression

[&larr; Back to course webpage](http://datakolektiv.com/app_direct/introdsnontech/)

![](../img/IntroMLPython_Head.png)

Feedback should be send to [goran.milovanovic@datakolektiv.com](mailto:goran.milovanovic@datakolektiv.com). 

These notebooks accompany the DATA SCIENCE SCHOOL :: Introduction to ML in Python course.

### Goran S. Milovanović, PhD
<b>DataKolektiv, Chief Scientist & Owner</b>

### Aleksandar Cvetković, PhD
<b>DataKolektiv, Consultant</b>

![](../img/DK_Logo_100.png)

## Setup

In [1]:
### --- Setup - importing the libraries

# - supress those annoying 'Future Warning'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


# - data
import numpy as np
import pandas as pd

# - os
import os

# - ml
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# - visualization
import matplotlib.pyplot as plt
import seaborn as sns



# - parameters
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
sns.set_theme()

# - rng
rng = np.random.default_rng()

# - plots
plt.rc("figure", figsize=(8, 6))
plt.rc("font", size=14)
sns.set_theme(style='white')

# - directory tree
data_dir = os.path.join(os.getcwd(), '_data')

In [2]:
# - loading the dataset
# - Kaggle: https://www.kaggle.com/datasets/uciml/iris
# - place it in your _data/ directory
iris_data = pd.read_csv(os.path.join(data_dir, 'Iris.csv'), index_col='Id')
iris_data.head(10)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa


In [3]:
# - counting the instances of each category
iris_data['Species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [4]:
# - info on the variables
iris_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 7.0+ KB


## Multinomial Logistic Regression

### Target: predict species from all continuous predictors

## Multinomial Logistic Regression

We use Multinomial Logistic Regression Model to predict the most probable category for the given observation $\textbf{x}$ with features $(x_1, x_2, \ldots, x_k)$. Assume that our target variable $y$ belongs to one of categories from the set $\{1, 2, \ldots, C\}$. In MNR we usually select one category as the referrence category; let that be the category $C$. Then, the probability that the target variable $y$ belongs to category $c = 1,\ldots,C-1$ is calculated via

$$P(y = c) = \frac{e^{\beta^{(c)}_1x_1 + \beta^{(c)}_2x_2 + \cdots + \beta^{(c)}_kx_k + n}}{1+\sum_{j=1}^{C-1}e^{\beta^{(j)}_1x_1 + \beta^{(j)}_2x_2 + \cdots + \beta^{(j)}_kx_k + n}},$$

and the probability that it belogns to the referrence category $C$ is 

$$P(y = C) = \frac{1}{1+\sum_{j=1}^{C-1}e^{\beta^{(j)}_1x_1 + \beta^{(j)}_2x_2 + \cdots + \beta^{(j)}_kx_k + n}},$$

where $\beta^{(j)}_1, \beta^{(j)}_2, \ldots, \beta^{(j)}_k,\ j=1,\ldots,C$ are the model's parameters for predictors and target variable categories, and $n$ is the intercept of the model.

After calculating all the probabilities $P(y = c),\ c=1,\ldots,C$ we predict the target variable as

$$\hat{y} = \textrm{argmax}_{c=1,\ldots,C}P(y=c).$$

The model is estimated by MLE (Maximum Likelihood Estimation). For each category $c$ - except for the referrence $C$, of course - we obtain a set of coefficients. Each model coefficient, in each category, tells us about the $\Delta_{odds}$ in favor of the target category, for a unit change of a predictor, in comparison with the baseline category, and given that everything else is kept constant.

In [5]:
### --- Preparing the variables 

# - feature matrix
X = iris_data.drop(columns='Species')
# - we append a constant column of ones to the feature matrix
X = sm.add_constant(X)
print(X[:10])


# - we impose the ordering to the categories of the target vector; the first category is the referrence category
cat_type = pd.CategoricalDtype(categories=["Iris-versicolor", "Iris-virginica", "Iris-setosa"], ordered=True)
y = iris_data['Species'].astype(cat_type)

    const  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
Id                                                                 
1     1.0            5.1           3.5            1.4           0.2
2     1.0            4.9           3.0            1.4           0.2
3     1.0            4.7           3.2            1.3           0.2
4     1.0            4.6           3.1            1.5           0.2
5     1.0            5.0           3.6            1.4           0.2
6     1.0            5.4           3.9            1.7           0.4
7     1.0            4.6           3.4            1.4           0.3
8     1.0            5.0           3.4            1.5           0.2
9     1.0            4.4           2.9            1.4           0.2
10    1.0            4.9           3.1            1.5           0.1


In [6]:
# - fitting the MNR model to the data; we use the Newton's Conjugate Gradient method as the optimizer to compute the
#models coefficients
mnr_model = sm.MNLogit(exog=X, endog=y).fit(method='ncg')
mnr_model.summary()

Optimization terminated successfully.
         Current function value: 0.039663
         Iterations: 26
         Function evaluations: 27
         Gradient evaluations: 27
         Hessian evaluations: 26


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                Species   No. Observations:                  150
Model:                        MNLogit   Df Residuals:                      140
Method:                           MLE   Df Model:                            8
Date:                Mon, 21 Nov 2022   Pseudo R-squ.:                  0.9639
Time:                        16:00:34   Log-Likelihood:                -5.9494
converged:                       True   LL-Null:                       -164.79
Covariance Type:            nonrobust   LLR p-value:                 7.056e-64
=======================================================================================
Species=Iris-virginica       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -42.6241     25.698     -1.659      0.097     -92.992       7.744
SepalLengthCm             -2.4650      2.394     -1.030      0.303      -7.158       2.227
SepalWidthCm              -6.6810      4.479     -1.492      0.136     -15.460       2.098
PetalLengthCm              9.4273      4.735      1.991      0.046       0.146      18.708
PetalWidthCm              18.2837      9.741      1.877      0.061      -0.808      37.375
------------------------------------------------------------------------------------------
Species=Iris-setosa       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   1.2685   3115.888      0.000      1.000   -6105.759    6108.296
SepalLengthCm           2.1956    882.047      0.002      0.998   -1726.585    1730.976
SepalWidthCm            6.4425    431.475      0.015      0.988    -839.233     852.118
PetalLengthCm         -10.8411    560.407     -0.019      0.985   -1109.220    1087.537
PetalWidthCm           -5.5832   1076.319     -0.005      0.996   -2115.130    2103.963
=======================================================================================
"""

In [7]:
# - confusion matrix for our model and given data; rows/columns are on par with the ordering of categorical variable
mnr_model.pred_table()

array([[49.,  1.,  0.],
       [ 1., 49.,  0.],
       [ 0.,  0., 50.]])

In [8]:
# - accuracy of the model
print(f'The accuracy of our model: {round(148/150, 4)}')

The accuracy of our model: 0.9867


In [9]:
# - model's prediction of probabilities; columns correspond to the ordering of categorical variable
mnr_model.predict()[:10]

array([[7.41122753e-09, 1.15691814e-35, 9.99999993e-01],
       [2.88109577e-07, 2.07898121e-32, 9.99999712e-01],
       [4.16727638e-08, 5.04109234e-34, 9.99999958e-01],
       [8.64262970e-07, 1.71937211e-31, 9.99999136e-01],
       [4.84673949e-09, 4.96320930e-36, 9.99999995e-01],
       [2.30189523e-08, 7.76396589e-34, 9.99999977e-01],
       [7.39465964e-08, 4.80628209e-33, 9.99999926e-01],
       [5.19827834e-08, 5.19878980e-34, 9.99999948e-01],
       [1.64479094e-06, 7.94019244e-31, 9.99998355e-01],
       [2.55926427e-07, 3.90502694e-33, 9.99999744e-01]])

In [10]:
# - model's prediction of categories; numbers correspond to the ordering of categorical variable
preds = np.argmax(mnr_model.predict(), axis=-1)
preds

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

### Multicolinearity in Multinomial Regression

In [11]:
### --- Step 1: recode categorical target variable to integer values, 
# - just in order to be able to run a multiple linear regression on the data:
y = y.cat.codes
y

Id
1      2
2      2
3      2
4      2
5      2
      ..
146    1
147    1
148    1
149    1
150    1
Length: 150, dtype: int8

In [12]:
### --- Step 2: produce a Multiple Linear Regression model for the data 
mnr_model = sm.OLS(exog=X, endog=y).fit()
mnr_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.571
Model:                            OLS   Adj. R-squared:                  0.559
Method:                 Least Squares   F-statistic:                     48.17
Date:                Mon, 21 Nov 2022   Prob (F-statistic):           1.02e-25
Time:                        16:00:36   Log-Likelihood:                -119.03
No. Observations:                 150   AIC:                             248.1
Df Residuals:                     145   BIC:                             263.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.4405      0.509     -0.866      0.388      -1.446       0.565
SepalLengthCm     0.0872      0.143      0.608      0.544      -0.196       0.371
SepalWidthCm      0.6832      0.149      4.586      0.000       0.389       0.978
PetalLengthCm    -0.4413      0.142     -3.117      0.002      -0.721      -0.161
PetalWidthCm      0.4198      0.235      1.789      0.076      -0.044       0.884
==============================================================================
Omnibus:                       28.857   Durbin-Watson:                   0.448
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                7.904
Skew:                          -0.222   Prob(JB):                       0.0192
Kurtosis:                       1.966   Cond. No.                         91.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
# --- Step 3: compute VIFs for the predictors
predictors = iris_data.columns.drop('Species')
predictors

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'], dtype='object')

In [14]:
# - appending the columns of ones to the predictors' data
model_frame_predictors = sm.add_constant(iris_data[predictors])

In [15]:
# - computing VIFs
vifs = [variance_inflation_factor(model_frame_predictors.values, i) for i in range(1, len(predictors)+1)]
vifs = np.array(vifs).reshape(1, -1)
vifs
pd.DataFrame(vifs, columns=predictors)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,7.103113,2.099039,31.397292,16.141564


## Multinomial Logistic Regression using scikit-learn

In [16]:
# - import scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [17]:
### --- Preparing the variables 

# - feature matrix
X = iris_data.drop(columns='Species').values

# - target variable
y = iris_data['Species']

In [18]:
### --- Fitting the logistic model to the numerical data
# - scikit-learn does not implement the referrence category automatically 
# - experiment with hyperparameters to produce more accurate model
log_reg = LogisticRegression(solver='newton-cg', penalty='none')
log_reg.fit(X, y)

LogisticRegression(penalty='none', solver='newton-cg')

In [19]:
# - coefficents of the model; rows correspond to the order of appearance of categories in the target variable
log_reg.coef_, log_reg.intercept_

(array([[  4.88501685,   7.84683378, -12.83914117,  -6.66236914],
        [ -1.20989943,  -0.58296936,   1.70486765,  -5.81189786],
        [ -3.67511742,  -7.26386442,  11.13427351,  12.47426701]]),
 array([  2.32878337,  20.15458005, -22.48336342]))

In [20]:
# - model's accuracy
round(log_reg.score(X, y), 4)

0.9867

In [21]:
# - predictions
y_pred = log_reg.predict(X)
y_pred[:10]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa'], dtype=object)

In [22]:
# - confusion matrix for the given data
# - rows/columns rows correspond to the order of appearance of categories in the target variable
confusion_matrix(y, y_pred)

array([[50,  0,  0],
       [ 0, 49,  1],
       [ 0,  1, 49]], dtype=int64)

Goran S. Milovanović & Aleksandar Cvetković

DataKolektiv, 2022/23.

[hello@datakolektiv.com](mailto:goran.milovanovic@datakolektiv.com)

![](../img/DK_Logo_100.png)

<font size=1>License: <a href="https://www.gnu.org/licenses/gpl-3.0.txt">GPLv3</a> This Notebook is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version. This Notebook is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details. You should have received a copy of the GNU General Public License along with this Notebook. If not, see <a href="http://www.gnu.org/licenses/">http://www.gnu.org/licenses/</a>.</font>